<a href="https://colab.research.google.com/github/gabrielefrn/qc-minicourse/blob/main/qc_minicourse_10abr24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum noise and error mitigation

Simulanting quantum noise

In [ ]:
!pip install numpy
!pip install git+https://github.com/qiboteam/qiboedu
!pip install qibo
!pip install qibojit

  Cloning https://github.com/qiboteam/qiboedu to /tmp/pip-req-build-lh9f7m6d
  Running command git clone --filter=blob:none --quiet https://github.com/qiboteam/qiboedu /tmp/pip-req-build-lh9f7m6d
  Resolved https://github.com/qiboteam/qiboedu to commit 57594cde061a007f0ee6015d05c7a95bd4627baa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.2/258.2 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.0/524.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 14.0 MB/s eta 0:00:00
  Created wheel for qiboedu: filename=qiboedu-0.1.0-py3-none-any.whl size=166

ERROR: Operation cancelled by user


In [3]:
import numpy as np
import matplotlib.pyplot as plt

import qibo
from qibo import Circuit, gates, hamiltonians

from qiboedu.scripts import plotscripts

In [4]:
#set qibo's backend
qibo.get_backend()

[Qibo 0.2.7|INFO|2024-04-10 17:42:36]: Using qibojit (numba) backend on /CPU:0
INFO:qibo.config:Using qibojit (numba) backend on /CPU:0


'qibojit (numba)'

In [5]:
#define a parametric circuit with layered architecture
nqubits = 3
nlayers = 3

#activate density matrix mode
c = Circuit(nqubits, density_matrix=True)

for l in range(nlayers):
  for q in range(nqubits):
    c.add(gates.RY(q=q, theta=0))
    c.add(gates.RZ(q=q, theta=0))
  for q in range(0, nqubits-1, 1):
    c.add(gates.CNOT(q0=q, q1=q+1))
c.add(gates.M(*range(nqubits)))

MeasurementResult(qubits=(0, 1, 2), nshots=0)

In [9]:
print(c.draw())
print(f"\n",c.summary())

nparams = len(c.get_parameters())
print(f"Number of parameters: {nparams}")

q0: ─RY─RZ─o───RY─RZ─o───RY─RZ─o───M─
q1: ─RY─RZ─X─o─RY─RZ─X─o─RY─RZ─X─o─M─
q2: ─RY─RZ───X─RY─RZ───X─RY─RZ───X─M─

 Circuit depth = 13
Total number of gates = 25
Number of qubits = 3
Most common gates:
ry: 9
rz: 9
cx: 6
measure: 1
Number of parameters: 18


In [ ]:
#fix the set of angles